In [410]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline

## Make a list of observed files

In [411]:
WeatherFilePaths = []
mydir = 'C:\Users\cflhxb\Dropbox\APSIMPotato\Observed\SubStorValidationData\weather'
for file in os.listdir(mydir):
    if file.endswith('.WTH'):
        WeatherFilePaths.append(os.path.join(mydir, file))

## Make a list of data frames to work through

In [465]:
FileNames = []
WeatherFiles = []
Headers = []
for File in WeatherFilePaths:
    FileNames.append(File.replace('C:\\Users\\cflhxb\\Dropbox\\APSIMPotato\\Observed\\SubStorValidationData\\weather\\','').replace('.WTH',''))
    Headers.append(pd.read_csv(File,sep='\s+',skiprows=2,nrows=1))
    WeatherFiles.append(pd.read_csv(File,sep='\s+',skiprows=4,converters={'@DATE': lambda x: str(x)}).dropna(how = 'all',axis=1).dropna(how='all'))
#put a location column into weather dataframes
for w in range(len(WeatherFilePaths)):
    WeatherFiles[w].ix[:,'@'] = Headers[w]['@'][0]

## Parse weird substore date into an actual date format

In [453]:
def Centuary(Y):
    if float(Y) < 20:
        return '20'
    else:
        return '19'
count = 0    
for w in WeatherFiles:
    Y = [str(w.ix[x,'@DATE'])[:2] for x in w.index]
    C = [Centuary(x) for x in Y]
    Year = [C[x]+Y[x] for x in range(len(Y))]
    w.ix[:,'FileName'] = FileNames[count]
    w.ix[:,'Year'] = Year
    w.ix[:,'DOY'] = [str(w.ix[x,'@DATE'])[2:] for x in w.index]
    w.ix[:,'Date'] = [datetime.datetime.strptime(w.Year[x]+'/'+w.DOY[x],'%Y/%j') for x in w.index]
    count+=1

## Get all of the met data into a single data frame for cleaning

In [455]:
ColumnHeaders = []
for w in WeatherFiles:
    for c in list(w.columns):
        ColumnHeaders.append(c)
Columns = set(ColumnHeaders)
AllMetData = pd.DataFrame(columns=Columns)
for w in WeatherFiles:
    AllMetData = AllMetData.append(w)

## Get all the met data clean

In [456]:
AllMetData.drop_duplicates(inplace=True)
AllMetData.set_index('FileName',inplace=True)
AllMetData.ix[:,'Day'] = [int(AllMetData.ix[x,'DOY']) for x in range(AllMetData.index.size)]
AllMetData.replace(to_replace='[A-z]',value='',inplace=True,regex=True)
AllMetData.replace(to_replace=-99,value=np.nan,inplace=True)
AllMetData.ix[:,'RAIN'] = pd.to_numeric(AllMetData.RAIN)
AllMetData.ix[:,'SRAD'] = pd.to_numeric(AllMetData.SRAD)
AllMetData.ix[:,'TMAX'] = pd.to_numeric(AllMetData.TMAX)
AllMetData.ix[:,'TMIN'] = pd.to_numeric(AllMetData.TMIN)
AllMetData.ix[:,'WIND'] = pd.to_numeric(AllMetData.WIND)
AllMetData.ix[:,'Tmean'] = (AllMetData.ix[:,'TMAX'] + AllMetData.ix[:,'TMIN'])/2

In [458]:
APSIMMetData = pd.DataFrame(columns = ['year','day','rain','maxt','mint','radn'],index = AllMetData.index)
APSIMMetData.ix[:,'year'] = AllMetData.ix[:,'Year']
APSIMMetData.ix[:,'day'] = AllMetData.ix[:,'Day']
APSIMMetData.ix[:,'rain'] = AllMetData.ix[:,'RAIN']
APSIMMetData.ix[:,'maxt'] = AllMetData.ix[:,'TMAX']
APSIMMetData.ix[:,'mint'] = AllMetData.ix[:,'TMIN']
APSIMMetData.ix[:,'radn'] = AllMetData.ix[:,'SRAD']
APSIMMetData.dropna(subset=['rain','maxt','mint','radn'],inplace=True)

## Get the header information into a tidy format

In [479]:
ColumnHeaders = []
count = 0
for h in Headers:
    h.ix[0,'FileName'] = FileNames[count]
    count += 1
for h in Headers:
    for c in list(h.columns):
        ColumnHeaders.append(c)        
Columns = set(ColumnHeaders)
AllHeaders = pd.DataFrame(columns=Columns)
for h in Headers:
    AllHeaders = AllHeaders.append(h)
AllHeaders.drop_duplicates(inplace=True)
AllHeaders.set_index('FileName',inplace=True)

## make files for each location with header data

In [481]:
for file in AllHeaders.index:
    TopLine = pd.DataFrame(data = ['[weather.met.weather]'])
    TopLine.ix[1,0] = 'Site = ' + file
    TopLine.ix[2,0] = 'Latitude=' + str(AllHeaders.ix[file,'INSI'])
    TopLine.ix[3,0] = 'Longitude=' + str(AllHeaders.ix[file,'LAT'])
    TopLine.ix[4,0] = 'tav=' +  str(AllMetData.ix[file,'Tmean'].mean())
    Fpctle = np.percentile(AllMetData.ix[file,'Tmean'].dropna().values,5)
    NFpctle = np.percentile(AllMetData.ix[file,'Tmean'].dropna().values,95)
    TopLine.ix[5,0] = 'amp=' + str(NFpctle - Fpctle)
    TopLine.ix[6,0] = 'year day rain maxt mint radn'
    TopLine.ix[7,0] = '() () (mm) (oC) (oC) (MJ/m2/d)'
    TopLine.to_csv('C:\Users\cflhxb\Dropbox\APSIMPotato\MET files\\' +file+ '.met',
                   header=False,index=False)

## Add the met data into each file

In [482]:
for file in AllHeaders.index:
    MetData = APSIMMetData.ix[file,:]
    MetData.to_csv('C:\Users\cflhxb\Dropbox\APSIMPotato\MET files\\' +file+ '.met',
                   header=False,index=False,mode='a',sep=' ')